In [3]:
from PIL import Image as pil_image
from PIL.ImageDraw import Draw

from os.path import isfile
import os

import time

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler

import cv2
from skimage import io, transform
from PIL import Image

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
train_df = pd.read_csv("../../data/humpback-whale-identification/train.csv")
train_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [5]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    
    y = onehot_encoded
    return y, label_encoder

In [6]:
y, le = prepare_labels(train_df['Id'])

In [7]:
transforms_train = transforms.Compose([transforms.Resize((128, 128)),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                       std=[0.229, 0.224, 0.225])])

transforms_test = transforms.Compose([transforms.Resize((128, 128)),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                       std=[0.229, 0.224, 0.225])])

In [8]:
class HumpBackWhaleDataset(Dataset):
    def __init__(self, datafolder, datatype='train', df=None, transform=None, y=None):
        self.datafolder = datafolder
        self.datatype = datatype
        self.y = y
        
        if self.datatype == 'train':
            self.df = df.values
            
        self.image_files_list = [s for s in os.listdir(datafolder)]
        self.transform = transform
        
    def __len__(self):
        return len(self.image_files_list)
    
    def __getitem__(self, idx):
        if self.datatype == 'train':
            img_name = os.path.join(self.datafolder, self.df[idx][0])
            label = self.y[idx]
            
        elif self.datatype == 'test':
            img_name = os.path.join(self.datafolder, self.image_files_list[idx])
            label = np.zeros((5005, ))
            
        image = Image.open(img_name).convert('RGB')
        image = self.transform(image)
        
        if self.datatype == 'train':
            return image, label
        elif self.datatype == 'test':
            return image, label, self.image_files_list[idx]

In [9]:
train_dataset = HumpBackWhaleDataset(datafolder='../../data/humpback-whale-identification/train/',
                                     datatype='train',
                                     df=train_df,
                                     transform=transforms_train,
                                     y=y)

test_dataset = HumpBackWhaleDataset(datafolder='../../data/humpback-whale-identification/test/',
                                    datatype='test',
                                    transform=transforms_test)
train_dataset[0][0].shape

torch.Size([3, 128, 128])

In [22]:
train_sampler = SubsetRandomSampler(list(range(len(os.listdir('../../data/humpback-whale-identification/train/')))))
valid_sampler = SubsetRandomSampler(list(range(len(os.listdir('../../data/humpback-whale-identification/test/')))))

batch_size = 512
num_workers = 20

train_loader = DataLoader(train_dataset, 
                          batch_size=batch_size, 
                          sampler=train_sampler, 
                          num_workers=num_workers)
test_loader = DataLoader(test_dataset, 
                         batch_size=32, 
                         num_workers=num_workers)

#next(iter(train_loader))[0].shape

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 128 x 128 ---> 124 x 124 
        self.conv1 = nn.Conv2d(3, 32, 7, padding=1)
        self.conv2_bn = nn.BatchNorm2d(32)
        # 124 x 124 ---> 62 x 62
        self.pool = nn.MaxPool2d(2, 2)
        
        # 62 x 62 ---> 62 x 62
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # 62 x 62 ---> 20 x 20
        self.pool2 = nn.AvgPool2d(3, 3)
        
        # flatten: filers * 20 * 20
        self.fc1 = nn.Linear(64 * 20 * 20, 1024)
        self.fc2 = nn.Linear(1024, 5005)

        self.dropout = nn.Dropout(0.5)        

    def forward(self, x):
        x = self.pool(F.relu(self.conv2_bn(self.conv1(x))))
        x = self.pool2(F.relu(self.conv2(x)))
        # batch_size, flatten image
        x = x.view(-1, 64 * 20 * 20)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [12]:
model_conv = Net()

criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(model_conv.parameters(), lr=0.01)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [14]:
model_conv.cuda()
n_epochs = 10
for epoch in range(1, n_epochs+1):
    print(time.ctime(), 'Epoch:', epoch)

    train_loss = []
    exp_lr_scheduler.step()

    for batch_i, (data, target) in enumerate(train_loader):
         #print(batch_i)
        data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output = model_conv(data)
        loss = criterion(output, target.float())
        train_loss.append(loss.item())

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}')

sub = pd.read_csv('../../data/humpback-whale-identification/sample_submission.csv')

model_conv.eval()
for (data, target, name) in test_loader:
    data = data.cuda()
    output = model_conv(data)
    output = output.cpu().detach().numpy()
    for i, (e, n) in enumerate(list(zip(output, name))):
        sub.loc[sub['Image'] == n, 'Id'] = ' '.join(le.inverse_transform(e.argsort()[-5:][::-1]))

sub.to_csv('basic_model.csv', index=False)

Thu Feb 21 06:12:42 2019 Epoch: 1


KeyboardInterrupt: 

In [32]:
from fastai.vision import *

data = ImageDataBunch.from_folder(path='../../data/humpback-whale-identification',
                                  train='../../data/humpback-whale-identification/train',
                                  valid_pct=0.2,
                                  size=(128, 128),
                                  num_workers=10,
                                  bs=512)

In [33]:
learn = Learner(data, model_conv)

In [34]:
learn.fit(1)

epoch,train_loss,valid_loss
1,0.557833,0.555605


In [3]:
def conv9x9(inchannel, outchannel, stride=1):
    """9x9 convolution with padding"""
    return nn.Conv2d(inchannel, outchannel, kernel_size=9, stride=stride, padding=1, bias=True)

def conv3x3(inchannel, outchannel, stride=1):
    """9x9 convolution with padding"""
    return nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=True)

def conv2x2(inchannel, outchannel, stride=1):
    """9x9 convolution with padding"""
    return nn.Conv2d(inchannel, outchannel, kernel_size=2, stride=stride, padding=1, bias=True)

In [4]:
class BasicBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv2x2(inchannel, outchannel, stride=2)
        self.conv2 = conv3x3(outchannel, outchannel, stride)
        self.conv3 = conv3x3(outchannel, outchannel, stride)
        self.relu = nn.ReLU(inplace=True)
        self.bn = nn.BatchNorm2d(outchannel)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(x)
        
        out = self.conv2(x)
        out = self.relu(x)
        
        out = self.conv3(x)
        out = self.relu(x)
        
        out = self.bn(x)
        
        return out

In [10]:
class BBModel(nn.Module):
    def __init__(self, block, blocks):
        super(BBModel, self).__init__()
        
        self.inchannel = 64
        
        self.conv1 = conv9x9(3, 64)       
        self.conv2 = conv3x3(64, 64)
        self.relu = nn.ReLU(inplace=True)
        self.bn = nn.BatchNorm2d(64)
        
        self.layer = self._make_layer(block, 64, blocks)
        
        self.pool_h = nn.MaxPool2d((1, 4))   
        self.pool_v = nn.MaxPool2d((4, 1))
        
        self.fc_h = nn.Linear(256, 16)
        self.fc_v = nn.Linear(256, 16)
        
        self.fc = nn.Linear(32,4)
        
        
    def _make_layer(self, block, outchannel, blocks, stride=1):
        layers = []
        for _ in range(blocks):
            layers.append(block(self.inchannel, outchannel))
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(x)
        
        out = self.conv2(x)
        out = self.relu(x)
        out = self.bn(x)
        
        out = self.layer(x)
        
        h = self.pool_h(out)
        h = torch.flatten(h)
        h = self.fc_h(h)
        h = self.relu(h)
        
        v = self.pool_v(out)
        v = torch.flatten(v)
        v = self.fc_v(v)
        v = self.relu(v)
        
        out = torch.concate((h, v), -1)
        out = self.fc(out)
        
        return out

In [11]:
model = BBModel(BasicBlock, 5)
model

BBModel(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): ReLU(inplace)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): ReLU(inplace)
      (bn): BatchNorm

In [14]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.032)

In [ ]:
epochs = 5
for e in range(epochs):
    running_loss = 0
    
    